In [1]:
import itertools

In [2]:
input_file = './data/dummy_data.csv'

In [3]:
global freq_itemsets, support, basket_count

freq_itemsets = []
support = 3

In [4]:
data = []
with open(input_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        tid = line.split(',')[0]
        items = line.replace('\n','').split(',')[1:]
        if len(items)>1:
            assert NotImplementedError
        data.append(tuple([tid, items[0]]))

data

[('1', 'apple'),
 ('1', 'carrot'),
 ('1', 'durian'),
 ('2', 'banana'),
 ('2', 'carrot'),
 ('2', 'edamame'),
 ('3', 'apple'),
 ('3', 'banana'),
 ('3', 'carrot'),
 ('3', 'edamame'),
 ('4', 'banana'),
 ('4', 'edamame')]

In [5]:
data_dict = dict()
for item in data:
    if item[0] not in data_dict.keys():
        data_dict[item[0]] = set()
        data_dict[item[0]].add(item[1])
    else:
        data_dict[item[0]].add(item[1])
        
data_dict

{'1': {'apple', 'carrot', 'durian'},
 '2': {'banana', 'carrot', 'edamame'},
 '3': {'apple', 'banana', 'carrot', 'edamame'},
 '4': {'banana', 'edamame'}}

In [6]:
basket = list(data_dict.values())
basket_count = len(basket)

for txn in basket:
    print(txn)

{'apple', 'durian', 'carrot'}
{'banana', 'carrot', 'edamame'}
{'banana', 'apple', 'carrot', 'edamame'}
{'banana', 'edamame'}


In [7]:
def create_count_dict(items, support_value):
    item_counts = {}
    for item in items:
        if item not in item_counts.keys():
            item_counts[item] = 1
        else:
            if item_counts[item] < support_value:
                item_counts[item] = item_counts[item] + 1
    return item_counts

def create_count_dict_tuple(chunk, support_value, candidate_tuples):
    tuple_counts = {}
    for basket in chunk:
        for candidate in candidate_tuples:
            if set(candidate).issubset(basket):
                if candidate in tuple_counts and tuple_counts[candidate] < support_value:
                    tuple_counts[candidate] += 1
                elif candidate not in tuple_counts:
                    tuple_counts[candidate] = 1
    return tuple_counts

def filter_by_support(count_dict, support_value, is_tuple):
    frequent_candidates = []
    for candidate, count in count_dict.items():
        if count >= support_value:
            frequent_candidates.append(candidate)
            freq_itemsets.append((candidate if is_tuple else tuple({candidate}), 1))
    return frequent_candidates


def get_frequent_candidate_tuples(frequent_items, size):
    candidates = []
    for item_x in frequent_items:
        for item_y in frequent_items:
            combined_set = tuple(sorted(set(item_x + item_y)))
            if len(combined_set) == size:
                if combined_set not in candidates:
                    previous_candidates = list(itertools.combinations(combined_set, size - 1))
                    flag = True
                    for candidate in previous_candidates:
                        if candidate not in frequent_items:
                            flag = False
                            break
                    if flag:
                        candidates.append(combined_set)
    return candidates


def get_frequent_candidate_tuples(frequent_items, size):
    candidates = []
    for item_x in frequent_items:
        for item_y in frequent_items:
            combined_set = tuple(sorted(set(item_x + item_y)))
            if len(combined_set) == size:
                if combined_set not in candidates:
                    previous_candidates = list(itertools.combinations(combined_set, size - 1))
                    flag = True
                    for candidate in previous_candidates:
                        if candidate not in frequent_items:
                            flag = False
                            break
                    if flag:
                        candidates.append(combined_set)
    return candidates


def find_frequent_candidates(chunk, chunk_support, frequent_items, size):
    if size == 2:
        candidates = list(itertools.combinations(sorted(frequent_items), 2))
    else:
        candidates = get_frequent_candidate_tuples(frequent_items, size)
    return filter_by_support(create_count_dict_tuple(chunk, chunk_support, candidates), chunk_support, True)


def apriori(basket):
    items = []
    
    for txn in basket:
        for item in txn:
            items.append(item)
    #frequent itemsets of size 1

    frequent_items = filter_by_support(create_count_dict(items, support), support, False)
    size = 2
    while len(frequent_items) != 0:
        frequent_items = find_frequent_candidates(basket, support, frequent_items, size)
        size += 1

    return freq_itemsets

In [8]:
%%time
apriori(basket)

CPU times: user 23 µs, sys: 18 µs, total: 41 µs
Wall time: 45.1 µs


[(('carrot',), 1),
 (('banana',), 1),
 (('edamame',), 1),
 (('banana', 'edamame'), 1)]